In [ ]:
!which python

# imports

In [ ]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from functools import partial, reduce
from typing import DefaultDict, Tuple, List, Dict
from torch import nn
import pandas as pd
from tqdm import tqdm
import pickle
import nlp_utils # our utils

In [ ]:
# !nvidia-smi

In [ ]:
cache_dir = nlp_utils.get_cache_dir()
device = "cuda" if torch.cuda.is_available() else "cpu"

# load Llava

In [ ]:
model_name = "llava-hf/llava-v1.6-mistral-7b-hf"

processor = LlavaNextProcessor.from_pretrained(model_name, cache_dir=cache_dir)
model = LlavaNextForConditionalGeneration.from_pretrained(model_name, torch_dtype=torch.float16, cache_dir=cache_dir)
model.generation_config.pad_token_id = processor.tokenizer.pad_token_id

model = model.to(device)

# load data

In [ ]:
df_data = nlp_utils.get_animals_data_df(limit_per_class=50)
print(len(df_data))
df_data.head()

In [ ]:
ids = np.random.randint(len(df_data), size=4)
image_paths = [df_data.image_path.iloc[i] for i in ids]
images = [Image.open(p) for p in image_paths]
Image.fromarray(np.concatenate([img.resize((256, 256)) for img in images], axis=1))

# run model on data

In [ ]:
image1 = Image.open(df_data.image_path[22])
image1

In [ ]:
image2 = Image.open(df_data.image_path[66])
image2

In [ ]:
question = 'What is common between these two images'
question = 'Who is the CEO of google?'
prompt = f"[INST] <image>\n{question} [/INST]"
inputs = processor(prompt, image2, return_tensors="pt").to(device)
output = model.generate(**inputs, max_new_tokens=100, output_hidden_states=True, return_dict_in_generate=True)
result_text = processor.decode(output['sequences'][0], skip_special_tokens=True)
result_text = result_text.split(prompt.replace('<image>', ' '))[1].strip()
print(f'Q: {question};\nA: {result_text}')

# extract number of layers

In [ ]:
all_module_names = [name for name, _ in model.named_modules()]
# print(all_module_names[:5])
lang_model_related_layers = [module_name for module_name in all_module_names if 'language_model.model.layers.' in module_name]
layers = list(set([e.split('language_model.model.layers.')[1].split('.')[0] for e in lang_model_related_layers]))
n_layers = max([int(e) for e in layers])
n_layers

# Save hidden states using my hooks

In [ ]:
import graphviz
from PIL import Image

def plot_model_graph(nodes, max_depth=3):
    dot = graphviz.Digraph()

    def add_edges(dot, nodes, max_depth=None):
        edges = set()
        for node in nodes:
            parts = node.split('.')
            for i in range(1, len(parts)):
                if max_depth is not None and i >= max_depth:
                    break
                parent = '.'.join(parts[:i])
                child = '.'.join(parts[:i+1])
                edges.add((parent, child))
        for edge in edges:
            dot.edge(edge[0], edge[1])
    
    def generate_graph(nodes, max_depth=None):
        dot = graphviz.Digraph()
        add_edges(dot, nodes, max_depth)
        return dot
    
    graph_with_depth = generate_graph(nodes, max_depth)
    file_path_depth = "./hierarchical_structure_graph_depth"
    graph_with_depth.render(file_path_depth, format='png', cleanup=True)
    img_path = file_path_depth +'.png'
    display(Image.open(img_path))

plot_model_graph(all_module_names)

In [ ]:
# layer_f_string = 'language_model.model.layers.{0}'
# layer_f_string = 'language_model.model.layers.{0}.self_attn'
# layer_f_string = 'language_model.model.layers.{0}.self_attn.q_proj'
# layer_f_string = 'language_model.model.layers.{0}.self_attn.k_proj'
# layer_f_string = 'language_model.model.layers.{0}.self_attn.v_proj'
# layer_f_string = 'language_model.model.layers.{0}.self_attn.o_proj'
# layer_f_string = 'language_model.model.layers.{0}.self_attn.rotary_emb'
# layer_f_string = 'language_model.model.layers.{0}.mlp'
# layer_f_string = 'language_model.model.layers.{0}.mlp.gate_proj'
# layer_f_string = 'language_model.model.layers.{0}.mlp.up_proj'
# layer_f_string = 'language_model.model.layers.{0}.mlp.down_proj'
# layer_f_string = 'language_model.model.layers.{0}.mlp.act_fn'
# layer_f_string = 'language_model.model.layers.{0}.input_layernorm'
layer_f_string = 'language_model.model.layers.{0}.post_attention_layernorm'

print('layer_f_string:', layer_f_string, end=2*'\n')


layer_names = [layer_f_string.format(layer_i) for layer_i in range(n_layers)] 
layer_names[:5]

In [ ]:
def save_activations(
        activations: DefaultDict,
        name: str,
        module: nn.Module,
        inp: Tuple,
        out: torch.Tensor) -> None:
    if name == 'language_model.model.layers.2.post_attention_layernorm':
        print('hello', len(activations[name]))
    activations[name].append(out.detach().cpu())

def get_module_by_name(module, access_string):
    names = access_string.split(sep='.')
    return reduce(getattr, names, module)

def set_hooks(model, activations_dict, module_names_to_hook):
    handles = []
    for name in module_names_to_hook:
        print('register_forward_hook to', name)
        module = get_module_by_name(model, name)
        hook = partial(save_activations, activations_dict, name)
        handle = module.register_forward_hook(hook)
        handles.append(handle)
    return handles

def remove_hooks(handles):
    for handle in handles:
        handle.remove()

def get_activations(activations_dict, layer_i):
    layer_str = layer_f_string.format(layer_i)
    activations = activations_dict[layer_str]
    return activations


# activations_dict = defaultdict(list)
# handles = set_hooks(model, activations_dict, layer_names)

In [ ]:
# remove_hooks(handles)

# get hidden states directly from the model using output_hidden_states

In [ ]:
question = 'What kind of animal is in the image? answer shortly'
prompt = f"[INST] <image>\n{question} [/INST]"
inputs = processor(prompt, image2, return_tensors="pt").to(device)
output = model.generate(**inputs, max_new_tokens=100, output_hidden_states=True, return_dict_in_generate=True)
result_text = processor.decode(output['sequences'][0], skip_special_tokens=True)
result_text = result_text.split(prompt.replace('<image>', ' '))[1].strip()
print(f'Q: {question};\nA: {result_text}')
print('# input tokens:', inputs['input_ids'].shape[1])
print('# output tokens:', output['sequences'].shape[1])

In [ ]:
image2.size

In [ ]:
output['hidden_states'][0][0].shape

In [ ]:
output['hidden_states'][0][0].shape

In [ ]:
"""
====================================
     hidden_states structure
====================================

time0
    layer_0
        (1, 2182, 4096)  # for the first response we are getting the hidden states of all positions up to 2182
    ....
    layer_32
        (1, 2182, 4096)
time1
    layer_0
        (1, 1, 4096)
    ....
    layer_32
        (1, 1, 4096)
...
timeT
    layer_0
        (1, 1, 4096)
    ....
    layer_32
        (1, 1, 4096)   
         
"""
print('')

In [ ]:
hidden_states = output['hidden_states']

time = 0       # 0 < time < seuqnce_length
layer = -1     # 0 < layer < 32
sample = 0     # batch size is 1
token_loc = 0  # 0 < token_loc <= time < seuqnce_length

embd = hidden_states[time][layer][sample]
embd.shape

# Project hidden states on the output embedding matrix

In [ ]:
hidden_states

In [ ]:
processor.tokenizer(['water'])

In [ ]:
time = 0
sample = 0
token_loc = -1
token_loc = -1

W = model.get_output_embeddings().weight  # the Unembedding matrix (#tokens, embedding_dim) = (32000, 4096)

for layer in range(28, n_layers+1):
    print('layer -', layer)
    embd = hidden_states[time][layer][sample][token_loc]

    logits_l = W @ embd
    closest_inds = torch.argsort(logits_l, descending=True)[:5]
    for i, ind in enumerate(closest_inds):
        most_probable_token = processor.decode(ind)
        print(f'rank {i} hidden state is closest to: {most_probable_token}')

# store hidden states of the entire dataset

In [ ]:

ddl_inference = defaultdict(list)
for _, row in tqdm(df_data.iterrows(), total=len(df_data)):
    image = Image.open(row.image_path)
    question = 'Describe the animal in the image?'
    prompt = f"[INST] <image>\n{question} [/INST]"
    inputs = processor(prompt, image, return_tensors="pt").to(device)
    output = model.generate(**inputs, max_new_tokens=100, output_hidden_states=True, return_dict_in_generate=True)
    # result_text = processor.decode(output['sequences'][0], skip_special_tokens=False)
    # result_text = result_text.split(prompt)[-1].strip()
    hidden_states = output['hidden_states']
    hidden_states = tuple(tuple(t.detach().cpu().numpy() for t in inner_tuple) for inner_tuple in hidden_states)
    input_ids = list(inputs['input_ids'][0].detach().cpu().numpy())      # [1, 733, 16289
    output_ids = list(output['sequences'][0].detach().cpu().numpy())     # [1, 733, 16289, ...

    ddl_inference['hidden_states'].append(hidden_states)
    ddl_inference['input_ids'].append(input_ids)
    ddl_inference['output_ids'].append(output_ids)
    

# train classifier on different hidden states

In [ ]:
def get_tokens(ids: List[int]):
    return [processor.tokenizer.decode(i) for i in ids]


def get_text(ids: List[int]):
    return processor.decode(ids)  


def print_data(input_ids, output_ids, hidden_states):
    input_ids                               # [1, 733, 16289
    input_tokens = get_tokens(input_ids)    # ['<s>', '[', 'INST', ..
    input_text = get_text(input_ids)        # '<s> [INST...
    
    output_ids                              # [1, 733, 16289, ...
    output_tokens = get_tokens(output_ids)  # ['<s>', '[', 'INST', ..
    output_text = get_text(output_ids)      # '<s> [INST...
    
    assert input_ids == output_ids[:len(input_ids)]
    assert input_tokens == output_tokens[:len(input_tokens)]
    assert input_text == output_text[:len(input_text)]
    assert len(hidden_states) == len(output_ids) - len(input_ids)
    
    hs = hidden_states[time][layer][sample][location]
    
    plt.figure()
    plt.imshow(image)
    plt.show()
    
    print(hs.shape)
    # hs = hs.detach().cpu()
    logits_l = W @ hs
    closest_inds = torch.argsort(logits_l, descending=True)[:1]
    for i, ind in enumerate(closest_inds):
        most_probable_token = processor.decode(ind)
        print(f'rank {i} hidden state is closest to: {most_probable_token}')

In [ ]:
print()

In [ ]:
# import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score


class_names = sorted(list(df_data.class_name.unique()))
labels = df_data.class_name.apply(lambda name: class_names.index(name)).to_list()
y = labels


# define grid
token_locs = list(range(-len(input_ids), 0))
token_locs = [-3]
layers = list(range(n_layers))
layers = list(range(n_layers-4, n_layers))

pbar = tqdm(total=len(token_locs) * len(layers))

time = 0
sample = 0

ddl = defaultdict(list)
for token_loc in token_locs:
    for layer in layers:
        X = np.array([hs[time][layer][sample][token_loc] for hs in ddl_inference['hidden_states']])
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        classifier = MLPClassifier(max_iter=200)
        
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
    
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)

        ddl['token_loc'].append(token_loc)
        ddl['layer'].append(layer)
        ddl['accuracy'].append(accuracy)
        pbar.update(1)
    
        # print("Accuracy:", accuracy)
        # print("Classification Report:\n", report)
pbar.close()

In [ ]:
df_acc = pd.DataFrame(ddl)
df_acc

In [ ]:
# with open('./training_results.pickle', 'wb') as f:
#     pickle.dump(ddl, f)

with open('./training_results.pickle', 'rb') as f:
    ddl_loaded = pickle.load(f)

In [ ]:
df_acc = pd.DataFrame(ddl_loaded)
df_acc

In [ ]:
token_locs = df_acc.token_loc.unique()
token_ids = np.array(input_ids)[token_locs]
tokens = processor.tokenizer.convert_ids_to_tokens(token_ids)

import seaborn as sns
plt.figure(figsize=(25, 8))
heatmap_data = df_acc.pivot(columns=["token_loc"], index="layer", values="accuracy")
ax = sns.heatmap(heatmap_data, annot=True, cmap="YlGnBu", )
ax.invert_yaxis()
plt.title('Accuracy Heatmap')
plt.xticks(np.array(range(len(tokens)))+0.5, tokens)

plt.show()

In [ ]:
"Answer the question using a single word or a phrase"

In [ ]:
df_acc

In [ ]:
df_temp = df_acc[df_acc.token_loc == -5]
x = df_temp.layer
y = df_temp.accuracy


plt.figure(figsize=(8, 5))
plt.title('Classifier Accuracy v.s. Hidden Layer Depth')
plt.grid(True)
plt.ylabel('Accuracy')
plt.xlabel('layer')
plt.plot(x, y, label='trained classifeir accuracy')
plt.axhline(1 / len(class_names), label='random guess', c='red')
plt.legend(loc='upper right')
plt.show()